# E-commerce Amazon Dataset — Initial Load

**Source:** Amazon Sale Report CSV  
**State:** Raw, unmodified  
**Scope:** Structural exposure only (no assumptions, no cleaning)

In [134]:
import pandas as pd
import numpy as np

data = pd.read_csv("e-com_data/Amazon Sale Report.csv")

print(data.columns)
print("Number of rows:", len(data))
data.head(3)

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')
Number of rows: 128975


/tmp/ipykernel_6673/3809125883.py:4: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("e-com_data/Amazon Sale Report.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN


In [135]:
def is_primary_key(cols):
    return data[cols].duplicated().sum()

print(
    "There are",
    len(data) - is_primary_key(['Order ID', 'Date', 'Status']),
    "unique values in Order ID, Date, Status combination"
)
print("There are", len(data), "rows in the dataset")
print("There are", data['index'].nunique(), "unique values in index column")


There are 120378 unique values in Order ID, Date, Status combination
There are 128975 rows in the dataset
There are 128975 unique values in index column


Index is a primary key for this table.

There is no other unique identifier derivable from the actual business fields.

Cardinality checks for individual columns and selected categorical fields.

Purpose:
- Identify low-cardinality categorical variables
- Expose structural constants, enums, and degenerate columns
- Detect columns unsuitable for modeling or requiring encoding


In [136]:
import sqlite3
import pandas as pd

print("Starting SQL Export...")

# 1. Create SQLite database
conn = sqlite3.connect("Ecom_Amazon.db")

# 2. Load dataframe into SQL
data.to_sql("ecom_data", conn, if_exists="replace", index=False)
print("Data successfully loaded into 'Ecom_Amazon.db'")

# 3. Cardinality / schema inspection via SQL

queries = {
    "Category cardinality": """
        SELECT COUNT(DISTINCT Category) AS unique_category_count FROM ecom_data;
    """,
    "Style cardinality": """
        SELECT COUNT(DISTINCT Style) AS unique_style_count FROM ecom_data;
    """,
    "SKU cardinality": """
        SELECT COUNT(DISTINCT SKU) AS unique_sku_count FROM ecom_data;
    """,
    "ASIN cardinality": """
        SELECT COUNT(DISTINCT ASIN) AS unique_asin_count FROM ecom_data;
    """,
    "Order count": """
        SELECT COUNT(DISTINCT "Order ID") AS distinct_orders FROM ecom_data;
    """,
    "Fulfilment values": """
        SELECT Fulfilment, COUNT(*) AS count
        FROM ecom_data
        GROUP BY Fulfilment;
    """,
    "Sales Channel values": """
        SELECT "Sales Channel ", COUNT(*) AS count
        FROM ecom_data
        GROUP BY "Sales Channel ";
    """,
    "Ship service level values": """
        SELECT "ship-service-level", COUNT(*) AS count
        FROM ecom_data
        GROUP BY "ship-service-level";
    """,
    "Courier Status values": """
        SELECT "Courier Status", COUNT(*) AS count
        FROM ecom_data
        GROUP BY "Courier Status";
    """,
    "Status values": """
        SELECT Status, COUNT(*) AS count
        FROM ecom_data
        GROUP BY Status;
    """,
    "Qty distribution": """
        SELECT Qty, COUNT(*) AS count
        FROM ecom_data
        GROUP BY Qty
        ORDER BY Qty;
    """,
    "B2B distribution (including NULL)": """
        SELECT 
            CASE WHEN B2B IS NULL THEN 'NULL' ELSE CAST(B2B AS TEXT) END AS B2B_value,
            COUNT(*) AS count
        FROM ecom_data
        GROUP BY B2B_value;
    """,
    "Size cardinality": """
        SELECT COUNT(DISTINCT Size) AS unique_size_count FROM ecom_data;
    """,
    "Promotion ID cardinality": """
        SELECT COUNT(DISTINCT "promotion-ids") AS unique_promo_ids FROM ecom_data;
    """,
    "fulfilled-by NaN count": """
        SELECT COUNT(*) AS null_count
        FROM ecom_data
        WHERE "fulfilled-by" IS NULL;
    """,
    "Unnamed: 22 distribution": """
        SELECT "Unnamed: 22", COUNT(*) AS count
        FROM ecom_data
        GROUP BY "Unnamed: 22";
    """
}

print("\n--- SQL Cardinality & Enum Checks ---")
for name, q in queries.items():
    # print(f"{name}")
    print(pd.read_sql(q, conn))

# 4. Close connection
conn.close()
print("\nSQL Pipeline Complete.")


Starting SQL Export...
Data successfully loaded into 'Ecom_Amazon.db'

--- SQL Cardinality & Enum Checks ---
   unique_category_count
0                      9
   unique_style_count
0                1377
Data successfully loaded into 'Ecom_Amazon.db'

--- SQL Cardinality & Enum Checks ---
   unique_category_count
0                      9
   unique_style_count
0                1377
   unique_sku_count
0              7195
   unique_asin_count
0               7190
   unique_sku_count
0              7195
   unique_asin_count
0               7190
   distinct_orders
0           120378
  Fulfilment  count
0     Amazon  89698
1   Merchant  39277
  Sales Channel    count
0      Amazon.in  128851
1     Non-Amazon     124
   distinct_orders
0           120378
  Fulfilment  count
0     Amazon  89698
1   Merchant  39277
  Sales Channel    count
0      Amazon.in  128851
1     Non-Amazon     124
  ship-service-level  count
0          Expedited  88615
1           Standard  40360
  Courier Status   coun

Same check done using Python

In [137]:
print("There are", data['Category'].nunique(), " unique values in Category column")
print("There are", len(data), "rows in the dataset")
print("There are", data['Style'].nunique(), " unique values in Style column")
print("There are", data['SKU'].nunique(), " unique values in SKU column")
print("There are", data['ASIN'].nunique(), " unique values in ASIN column")
print("There are", data['Order ID'].nunique(), "distinct orders in the dataset")
print("There are", data['Fulfilment'].nunique(), " unique values in Fulfilment column")
print("There are", data['Sales Channel '].nunique(), " unique values in Sales Channel column")
print("There are", data['ship-service-level'].nunique(), " unique values in ship-service-level column")
print(data['Sales Channel '].unique())
print(data['ship-service-level'].unique())
print("status column")
print(data['Courier Status'].nunique())
print(data['Courier Status'].unique())
print(data['Status'].nunique())
print(data['Status'].unique())
print(data['Qty'].nunique())
print(data['Qty'].unique())
print(data['B2B'].nunique())
print(data['B2B'].unique())
print(data['Category'].unique())
print("size ",data['Size'].nunique())
print(data['Size'].unique())
print("Number of unique promotion ids are ", data['promotion-ids'].nunique())
print(data['fulfilled-by'].isna().sum(), "NaN values are there")
print(data['B2B'].value_counts(dropna=False))
print("last column")
print(data['Unnamed: 22'].value_counts(dropna=False))

There are 9  unique values in Category column
There are 128975 rows in the dataset
There are 1377  unique values in Style column
There are 7195  unique values in SKU column
There are 7190  unique values in ASIN column
There are 120378 distinct orders in the dataset
There are 2  unique values in Fulfilment column
There are 2  unique values in Sales Channel column
There are 2  unique values in ship-service-level column
['Amazon.in' 'Non-Amazon']
['Standard' 'Expedited']
status column
3
[nan 'Shipped' 'Cancelled' 'Unshipped']
13
['Cancelled' 'Shipped - Delivered to Buyer' 'Shipped'
 'Shipped - Returned to Seller' 'Shipped - Rejected by Buyer'
 'Shipped - Lost in Transit' 'Shipped - Out for Delivery'
 'Shipped - Returning to Seller' 'Shipped - Picked Up' 'Pending'
 'Pending - Waiting for Pick Up' 'Shipped - Damaged' 'Shipping']
10
[ 0  1  2 15  3  9 13  5  4  8]
2
[False  True]
['Set' 'kurta' 'Western Dress' 'Top' 'Ethnic Dress' 'Bottom' 'Saree'
 'Blouse' 'Dupatta']
size  11
['S' '3XL' 'XL

Relational consistency checks across product-identifying columns.

Purpose:
- Test whether identifier fields are hierarchically well-defined
- Verify one-to-many vs many-to-many relationships
- Detect semantic leakage between SKU, Style, ASIN, and Category
- Establish whether columns can be safely treated as categorical keys or bundles

In [138]:
sku_style_counts = (
    data[['SKU', 'Style']]
    .dropna()
    .groupby('SKU')['Style']
    .nunique()
)

print((sku_style_counts > 1).any())

ASIN_style_counts = (
    data[['ASIN', 'Style']]
    .dropna()
    .groupby('ASIN')['Style']
    .nunique()
)

print((ASIN_style_counts > 1).any())
print(ASIN_style_counts[ASIN_style_counts > 1])

cate_asin_counts = (
    data[['Category', 'ASIN']]
    .dropna()
    .groupby('Category')['ASIN']
    .nunique()
)

print((cate_asin_counts > 1).any())


False
True
ASIN
B091GNW3B1    2
Name: Style, dtype: int64
True


This validates that the (SKU, Style) mapping is distinct with Style being a broader classification and SKU being sub-classification. But, same is not true for ASIN. ASIN is a different categorisation.

Column inventory and semantic grouping.

- index  
  Synthetic primary key.

- Order ID (Ini)  
  Order-level identifier.

- Date (Ini)  
  Order date.

- Status (Pres)  
  Current order state.

- Fulfilment (Ini)  
  Amazon or Merchant.

- Sales Channel (Ini)  
  Amazon or non-Amazon.

- ship-service-level (Ini)  
  Standard or Expedited.

- Style (Ini)  
  Parent category of SKU for product identification.

- SKU (Ini)  
  Sub-category of Style.

- Category  
  Broad product category; no hierarchical descendants.

- Size  
  Product size attribute.

- ASIN (Ini)  
  Product categorisation similar to SKU, with one exception.

- Courier Status  
  Enum: {NaN, Shipped, Cancelled, Unshipped}.

- Economic bundle (Ini)  
  {Qty, currency, Amount}  
  Monetary semantics; not functionally dependent on Qty.

- Location bundle (Ini)  
  {ship-city, ship-state, ship-postal-code, ship-country}.

- promotion-ids (Ini)  
  High-cardinality categorical field (5,787 unique values).

- B2B  
  Boolean flag.

- fulfilled-by  
  Easy Ship or NaN.

- Unnamed: 22  
  Degenerate column; values ∈ {False, NaN}.


This dataset represents apparel purchases.

Evidence:
- Presence of Style and Size attributes
- Category values correspond to clothing items


Speculation.

Status and Courier Status appear to encode overlapping shipment state.
Their relationship may depend on the fulfilment/service provider.
This analysis tests whether their joint behavior differs across Fulfilment values.

In [139]:

data['Status'] = (
    data['Status']
    .astype(str)
    .str.strip()
    .str.lower()
)
data['Courier Status'] = (
    data['Courier Status']
    .astype(str)
    .str.strip()
    .str.lower()
)

In [140]:
courier_status_by_fulfilment = (
    data
    .groupby("Fulfilment")["Courier Status"]
    .value_counts(dropna=False)
)

print(courier_status_by_fulfilment)

single_word = data[
    data["Status"].notna() &
    (~data["Status"].str.contains(" "))
]
print(
    single_word
    .groupby(["Fulfilment", "Status"])["Courier Status"]
    .value_counts(dropna=False)
)

Fulfilment  Courier Status
Amazon      shipped           77606
            unshipped          6157
            cancelled          5935
Merchant    shipped           31881
            nan                6872
            unshipped           524
Name: count, dtype: int64
Fulfilment  Status     Courier Status
Amazon      cancelled  cancelled          5840
                       unshipped          5631
            pending    unshipped           403
                       shipped              10
                       cancelled             2
            shipped    shipped           77596
                       unshipped           115
                       cancelled            93
            shipping   unshipped             8
Merchant    cancelled  nan                6861
            pending    unshipped           243
Name: count, dtype: int64


This shows it and now we can make seperate tables.

In [179]:
for f in data["Fulfilment"].unique():
    print(f"\n=== Fulfilment = {f} ===")
    m = pd.crosstab(
        data.loc[data["Fulfilment"] == f, "Status"],
        data.loc[data["Fulfilment"] == f, "Courier Status"].fillna("__NaN__"),
        dropna=False
    )
    print(m)



=== Fulfilment = Merchant/Easy Ship ===
Courier Status                 Operation Finished  shipped  unshipped
Status                                                               
cancelled                                    6861        0          0
pending                                         0        0        243
pending - waiting for pick up                   0        0        281
shipped - damaged                               0        1          0
shipped - delivered to buyer                    8    28761          0
shipped - lost in transit                       0        5          0
shipped - out for delivery                      0       35          0
shipped - picked up                             0      973          0
shipped - rejected by buyer                     0       11          0
shipped - returned to seller                    3     1950          0
shipped - returning to seller                   0      145          0

=== Fulfilment = Amazon ===
Courier Status  canc

## Interpretation After Fulfillment Segregation

After separating orders by **Fulfillment type**, we observe that the **status hierarchy and transitions are fulfillment-dependent**, while remaining internally consistent within each table.

### Merchant Fulfillment

The Merchant table is largely **clean and self-consistent**:

- `Shipped` and its sub-states form a clear terminal chain.
- `Cancelled` appears only where shipment was not finalized.
- A small number of non-cancelled rows with `Courier Status = NaN` likely reflect **pre-assignment or missing courier metadata**, rather than contradictory states.

Overall, Merchant fulfillment exhibits **minimal ambiguity** in state transitions.

### Amazon Fulfillment

The Amazon table shows a **compressed state space** with fewer observable transitions:

- Orders largely separate into:
  - `Cancelled` (early-stage)
  - `Shipped` (dominant terminal state)
- Mixed states such as `Shipped + Unshipped` occur but are **rare**  
  (~115 out of ~77,700; ≈ 0.15%).

These low-frequency overlaps are best interpreted as:
- **system lag**,  
- **asynchronous updates**, or  
- **transitional states** where confirmation has occurred on one side but not yet propagated.

### Key Observations

- Status combinations with zero or near-zero counts appear to be **structurally forbidden**, not missing data.
- Cancellation behavior differs in *representation* across fulfillment types, but remains internally consistent within each table.
- Amazon fulfillment effectively collapses multiple intermediate steps into fewer visible states, reducing dimensionality.

### Practical Implication

Each fulfillment type should be analyzed **independently**, with:
- its own valid state hierarchy,
- its own interpretation of `NaN` and transitional states,
- and no assumption that all status combinations are meaningful across fulfillers.


### A theory based on the observed numbers (Data-Driven)

The observed counts are consistent with the following numerical behavior:

> When an order is cancelled, **only the fields corresponding to steps that are still in progress change to `Cancelled`**.  
> Fields corresponding to steps already completed **remain unchanged**.

Here, `Status` and `Courier Status` are treated purely as **separate recorded stages**, not as a single unified lifecycle.

This is sufficient to explain:
- why `Status = Shipped` can co-occur with `Courier Status = Cancelled`,
- why early cancellations appear as `Courier Status = NaN` with `Status = Cancelled`,
- and why certain state combinations never appear in the tables.

No additional structure is assumed beyond what is directly visible in the counts.

We decide to fill the Nan in courier status by 'Operation Finished' because everytime it occurs (cancelled/delivered/returned), is when operation has been finished. 

In [ ]:
data['Courier Status'].replace('nan', 'operation finished', inplace=True)

Now, we will make one column using this theory/interpretation for the actual status. We will name it 'Cancelled, Shipping, Shipped, Delivered, Returned'.

In [ ]:
for f in data["Fulfilment"].unique():
    print(f"\n=== Fulfilment = {f} ===")
    m = pd.crosstab(
        data.loc[data["Fulfilment"] == f, "Status"],
        data.loc[data["Fulfilment"] == f, "Courier Status"].fillna("operation finished"),
        dropna=False
    )
    print(m)


=== Fulfilment = Merchant ===
Courier Status                 Operation Finished  shipped  unshipped
Status                                                               
cancelled                                    6861        0          0
pending                                         0        0        243
pending - waiting for pick up                   0        0        281
shipped - damaged                               0        1          0
shipped - delivered to buyer                    8    28761          0
shipped - lost in transit                       0        5          0
shipped - out for delivery                      0       35          0
shipped - picked up                             0      973          0
shipped - rejected by buyer                     0       11          0
shipped - returned to seller                    3     1950          0
shipped - returning to seller                   0      145          0

=== Fulfilment = Amazon ===
Courier Status  cancelled  shi

In [144]:
data['status_info'] = 'active'
data['is_operational_loss'] = 0

s = data['Status'].fillna('')
c = data['Courier Status'].fillna('')

flags = {
    'cancelled': s.str.contains(r'\bcancel', regex=True),
    'pending': s.str.contains(r'\bpend', regex=True),
    'damaged': s.str.contains(r'\bdamag|\blost', regex=True),
    'returned': s.str.contains(r'\breturn|\breject', regex=True),
    'delivered': s.str.contains(r'\bdeliver', regex=True),
    'out_for_delivery': s.str.contains(r'\bout for delivery', regex=True),
    'picked_up': s.str.contains(r'\bpicked up', regex=True),
    'shipped': s.str.contains(r'\bship', regex=True),
    'shipping': s.str.contains(r'\bshipping\b', regex=True),

    'courier_cancelled': c.str.contains(r'\bcancel', regex=True)
}

is_merchant = data['Fulfilment'] == 'Merchant'

# cancelled
mask = is_merchant & flags['cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['cancelled', 1]

# damaged / lost
mask = is_merchant & flags['damaged']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['lost', 1]

# returned / rejected / returning
mask = is_merchant & flags['returned']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['returned', 1]

# delivered
mask = is_merchant & flags['delivered']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['delivered', 0]

# out for delivery OR picked up → shipped
mask = is_merchant & (flags['out_for_delivery'] | flags['picked_up'])
data.loc[mask, ['status_info', 'is_operational_loss']] = ['shipped', 0]

# pending → unshipped
mask = is_merchant & flags['pending']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['unshipped', 0]


is_amazon = data['Fulfilment'] == 'Amazon'

# cancelled (explicit)
mask = is_amazon & flags['cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['cancelled', 1]

# pending + courier cancelled → cancelled + loss
mask = is_amazon & flags['pending'] & flags['courier_cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['cancelled', 1]

# pending + courier not cancelled → pending
mask = is_amazon & flags['pending'] & ~flags['courier_cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['pending', 0]

# shipped + courier cancelled → cancelled + loss
mask = is_amazon & flags['shipped'] & flags['courier_cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['cancelled', 1]

# shipped + courier not cancelled → shipped
mask = is_amazon & flags['shipped'] & ~flags['courier_cancelled']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['shipped', 0]

# shipping → unshipped
mask = is_amazon & flags['shipping']
data.loc[mask, ['status_info', 'is_operational_loss']] = ['unshipped', 0]



In [180]:
print(pd.crosstab(
    [data['Fulfilment'], data['status_info']],
    data['is_operational_loss']
).sum().sum())
print(len(data))
pd.crosstab(
    [data['Fulfilment'], data['status_info']],
    data['is_operational_loss']
)


128969
128969


is_operational_loss                 0      1
Fulfilment         status_info              
Amazon             cancelled        0  11563
                   pending        413      0
                   shipped      77708      0
                   unshipped        8      0
Merchant/Easy Ship cancelled        0   6861
                   delivered    28769      0
                   lost             0      6
                   returned         0   2109
                   shipped       1008      0
                   unshipped      524      0

In [146]:
# a check to see if Fulfilment is same as fulfilled by
print(data[data["fulfilled-by"].isna()]['Fulfilment'].value_counts())

print(data[data["fulfilled-by"]=='Easy Ship']['Fulfilment'].value_counts())
print(data[data["Fulfilment"]=='Merchant']['fulfilled-by'].value_counts())

print(data[data["Fulfilment"]=='Amazon']['fulfilled-by'].value_counts())

Fulfilment
Amazon    89698
Name: count, dtype: int64
Fulfilment
Merchant    39277
Name: count, dtype: int64
fulfilled-by
Easy Ship    39277
Name: count, dtype: int64
Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)


Interpretation:
1) When 'sales Channel' = 'Non-Amazon', then fulfilment = Amazon & fulflled-by = NaN
2) When 'sales Channel' = 'Non-Amazon' and 'Fulfilment' = 'Merchant', then 'fulfilled-by' = 'Easy-Ship'.
3) We change all Fulfilment = Merchant to Merchant/Easy Ship and then drop the fulfilled-by column.

In [147]:
data.drop(columns=['fulfilled-by'], inplace=True)
data.loc[data['Fulfilment']=='Merchant', 'Fulfilment'] = 'Merchant/Easy Ship'

turning all NaNs in promotion ids to 'None'.

In [148]:
data['promotion-ids'].fillna('None', inplace=True)

a check to see if all nans are co-occuring in 
ship-city                  33
ship-state                 33
ship-postal-code           33
ship-country               33

In [149]:
data[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']] = data[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']].fillna('Unknown')

This confrimed that all nans in ship-city                  33
ship-state                 33
ship-postal-code           33
ship-country               33 are co-occuring. we will replace all those with 'Unknown'

a check for nans in each column

In [150]:
print(data.isna().sum())

index                      0
Order ID                   0
Date                       0
Status                     0
Fulfilment                 0
Sales Channel              0
ship-service-level         0
Style                      0
SKU                        0
Category                   0
Size                       0
ASIN                       0
Courier Status             0
Qty                        0
currency                7795
Amount                  7795
ship-city                  0
ship-state                 0
ship-postal-code           0
ship-country               0
promotion-ids              0
B2B                        0
Unnamed: 22            49050
status_info                0
is_operational_loss        0
dtype: int64


Do NaNs in Amount and Currency occur together?

In [151]:
# 1. focus slice
z1 = data[(data["currency"].notna()) & (data["Amount"].isna())].copy()
z2 = data[(data["currency"].isna()) & (data["Amount"].isna())].copy()
z3 = data[(data["currency"].isna())].copy()
print(len(z1), len(z2), len(z3))

0 7795 7795


The above results show that the instances of NaN in currency and amount occur together.

In [181]:
print(
    data.loc[
        (data['Qty'] > 0) & (data['Status'] == 'cancelled'),
        'Courier Status'
    ].value_counts(dropna=False)
)


Courier Status
unshipped    5631
Name: count, dtype: int64


This shows that when `Status = Cancelled` but `Qty > 0`, the courier status is `unshipped`.
This indicates cancellation before courier operations begin.

In [184]:
print(
    data.loc[
        (data['Courier Status'] == 'cancelled') |
        (data['Courier Status'] == 'operation finished'),
        'Qty'
    ].value_counts(dropna=False)
)

print(data.loc[data['Qty'] == 0, 'Courier Status'].value_counts(dropna=False))
print(data.loc[data['Qty'] > 0, 'Courier Status'].value_counts(dropna=False))
print(
    data.loc[
        (data['Courier Status'] == 'cancelled') |
        (data['Courier Status'] == 'operation finished'),
        'Qty'
    ].value_counts(dropna=False)
)


Qty
0    5932
Name: count, dtype: int64
Courier Status
Operation Finished    6872
cancelled             5932
Name: count, dtype: int64
Courier Status
shipped      109484
unshipped      6681
Name: count, dtype: int64
Qty
0    5932
Name: count, dtype: int64


Observations:

- `Qty = 0` is always accompanied by:
  - `Courier Status = Cancelled` (Amazon)
  - or `Courier Status = Operation Finished` (Merchant)

- `Operation Finished` appears only for Merchant fulfilment.
- `Cancelled` courier status is used exclusively for Amazon fulfilment.

Interpretation:

- `Qty = 0` indicates that the courier operation has terminated.
- This typically corresponds to cancellation, and occasionally delivery or return.
- All cancelled orders eventually set `Qty = 0`.
- `Qty > 0` always corresponds to either `Shipped` or `Unshipped`.


In [185]:

print(
    data.loc[
        (data['Courier Status'] == 'cancelled') |
        (data['Courier Status'] == 'operation finished'),
        'Qty'
    ].value_counts(dropna=False)
)
print(
    data.loc[
        (data['Status'] == 'cancelled') &
        (data['Courier Status'] != 'cancelled') &
        (data['Courier Status'] != 'operation finished'),
        'Qty'
    ].value_counts(dropna=False)
)


Qty
0    5932
Name: count, dtype: int64
Qty
0    6861
1    5605
2      26
Name: count, dtype: int64


The above observation confirms the theory:

`Qty = 0` is triggered only when courier operations are halted mid-operation,
causing the courier status to change to `Cancelled` or `Operation Finished`.

Thus, `Qty = 0` represents a terminal courier state, not missing quantity.


Now, we will study the presence of NaNs in Amount and currency.

In [155]:
print(data[(data["Qty"]==0) & (data["Status"] != "cancelled")].shape[0])
print(data[(data["Qty"]==0) & (data["Status"] != "cancelled") & (data["Amount"].isna())].shape[0])
# z = data[(data["Qty"]==0) & (data["Status"] != "Cancelled")].copy()
# print((z['Amount'].isna()).sum().sum())

106
106


They do co-occur when status is not 'cancelled'.

In [156]:
print(data[(data["Qty"]==0) & (data["Courier Status"] != "cancelled")].shape[0])
print(data[(data["Qty"]==0) & (data["Courier Status"] != "cancelled") & (data["Amount"].isna())].shape[0])

6872
1736


but same is not valid for courier status.

In [157]:
z = data[(data["Amount"]<0) ].copy()
print(len(z))
z = data[(data["Amount"]==0) ].copy()
print(len(z))
print(data["Amount"].dtype)

0
2343
float64


Amount is never negative and many times has 0 as an entry.

### Relationship between `Amount = NaN`, `Status`, and `Qty`

We now examine how instances of `Amount = NaN` relate to combinations of
`Qty`, `Status`, and `Courier Status`.


In [158]:
print(data.loc[(data['Qty']==0) & (data['Status'] != 'cancelled')]['Amount'].value_counts(dropna=False))

print((data.loc[(data['Amount'].isna()) & (data['Courier Status'] == 'unshipped')])['Qty'].value_counts(dropna=False))

Amount
NaN    106
Name: count, dtype: int64
Qty
1     115
2       4
15      1
9       1
13      1
4       1
3       1
Name: count, dtype: int64


The above outputs show that there exist rows where `Amount = NaN`
and `Courier Status = unshipped`, with strictly positive values of `Qty`.


In [159]:
print((data.loc[(data['Qty']!=0) & (data['Amount'].isna())])[['Status', 'Courier Status']].value_counts(dropna=False))

Status     Courier Status
shipped    unshipped         115
shipping   unshipped           8
cancelled  unshipped           1
Name: count, dtype: int64


For rows where `Qty ≠ 0` and `Amount = NaN`,
the observed `(Status, Courier Status)` combinations are:

- (`shipped`, `unshipped`)
- (`shipping`, `unshipped`)
- (`cancelled`, `unshipped`)


In [160]:
print(
    data.loc[
        (data['Status'] == 'shipped') &
        (data['Courier Status'] == 'unshipped'),
        'Qty'
    ].value_counts(dropna=False)
)


Qty
1     106
2       4
15      1
9       1
13      1
4       1
3       1
Name: count, dtype: int64


In [161]:
print(
    data.loc[
        (data['Status'] == 'shipping') &
        (data['Courier Status'] == 'unshipped'),
        'Qty'
    ].value_counts(dropna=False)
)


Qty
1    8
Name: count, dtype: int64


The identical `Qty` distributions observed above confirm that
rows with (`shipped`, `unshipped`) and (`shipping`, `unshipped`)
correspond exactly to the `Qty ≠ 0` & `Amount = NaN` cases.


At this point, two explanations are possible and are stated without preference:

1. When `Qty ≠ 0` and `(Status, Courier Status)` is one of
   (`shipped`, `unshipped`) or (`shipping`, `unshipped`),
   `Amount` is recorded as NaN.

2. Alternatively, when `Amount` is NaN, either `Qty` later becomes `0`
   or remains positive, and `Courier Status` transitions accordingly
   to terminal or non-terminal states.

No causal direction is asserted here.


In [162]:
print(
    data.loc[
        (data['Qty'] != 0) &
        (data['Amount'].isna()),
        ['Status', 'Courier Status', 'Fulfilment', 'Sales Channel ']
    ].value_counts(dropna=False)
)


Status     Courier Status  Fulfilment  Sales Channel 
shipped    unshipped       Amazon      Non-Amazon        115
shipping   unshipped       Amazon      Non-Amazon          8
cancelled  unshipped       Amazon      Non-Amazon          1
Name: count, dtype: int64


All rows with `Qty ≠ 0` and `Amount = NaN` have:

- `Fulfilment = Amazon`
- `Sales Channel = Non-Amazon`


In [163]:
print(
    data.loc[
        (data['Fulfilment'] == 'Amazon') &
        (data['Sales Channel '] == 'Non-Amazon'),
        ['Qty', 'Amount']
    ].value_counts(dropna=False)
)


Qty  Amount
1    NaN       115
2    NaN         4
3    NaN         1
4    NaN         1
9    NaN         1
13   NaN         1
15   NaN         1
Name: count, dtype: int64


So, all 124 entries which have qty!=0 and amount NaN, have fulfilment = amazon and sales channel non-amazon and which this fulfilment combination is present all amount entries are naN (124). This equivalence is now found, and we can devise to a method to fix it.

Now, we need to uncover the reason for amount being 0. This is apparently not same as amount being nan.

In [164]:
print((data.loc[ (data['Amount']==0)])[[ 'Qty','Courier Status', 'Fulfilment', 'Sales Channel ']].value_counts(dropna=False))

Qty  Courier Status  Fulfilment          Sales Channel 
1    shipped         Amazon              Amazon.in         1518
                     Merchant/Easy Ship  Amazon.in          799
     unshipped       Merchant/Easy Ship  Amazon.in           17
                     Amazon              Amazon.in            9
Name: count, dtype: int64


Now, that we have interpreted the presence of amount = NaN, we can plan for its imputation. Below are a few plans,
1) When Qty=0, put amount=0,doest matter. when qty!=0, then just check for similar product labels and copy the corresponding values (but first check that amount is Qty multiplied or not).

In [165]:
data.loc[(data["Qty"]==0) & (data['currency'].isna()), ['Amount','currency']]=[0,'none']

In [166]:
mask_fix = (
    (data['Qty'] != 0) &
    (data['currency'].isna())
)

keys = ['SKU', 'ASIN', 'Category', 'Style']

ref = (
    data
    .loc[data['Amount'].notna(), keys + ['Amount','currency']]
    .drop_duplicates(subset=keys)
)

data = data.merge(
    ref,
    on=keys,
    how='left',
    suffixes=('', '_ref')
)

mask_fix = (data['Qty'] != 0) & (data['currency'].isna())

data.loc[mask_fix & data['Amount_ref'].notna(), 'Amount'] = \
    data.loc[mask_fix & data['Amount_ref'].notna(), 'Amount_ref']

data.loc[mask_fix & data['currency_ref'].notna(), 'currency'] = \
    data.loc[mask_fix & data['currency_ref'].notna(), 'currency_ref']

data.drop(columns=['Amount_ref', 'currency_ref'], inplace=True)


Now there are only 10 amount = nans left out of total 128975 rows in the data.

delete duplicate rows presentt

In [167]:
dup_rows = data.duplicated(['Order ID', 'ASIN'], keep=False)
dup_rows.sum()


14

In [168]:
print(len(data))
data = data.drop_duplicates(
    subset=[c for c in data.columns if c != 'index']
)
print(len(data))

128975
128969


In [169]:
print(data.isna().sum())

index                      0
Order ID                   0
Date                       0
Status                     0
Fulfilment                 0
Sales Channel              0
ship-service-level         0
Style                      0
SKU                        0
Category                   0
Size                       0
ASIN                       0
Courier Status             0
Qty                        0
currency                  10
Amount                    10
ship-city                  0
ship-state                 0
ship-postal-code           0
ship-country               0
promotion-ids              0
B2B                        0
Unnamed: 22            49048
status_info                0
is_operational_loss        0
dtype: int64


Now, lets check ship-service-level entries and what they mean for other columns.

In [170]:
data.loc[(data['Fulfilment']=='Merchant/Easy Ship')]['ship-service-level'].value_counts(dropna=False)

ship-service-level
Standard    39277
Name: count, dtype: int64

In [171]:
print(data['ship-country'].value_counts(dropna=False))
print(data['currency'].value_counts(dropna=False))

ship-country
IN         128936
Unknown        33
Name: count, dtype: int64
currency
INR     121282
none      7677
NaN         10
Name: count, dtype: int64


dropping ship-country and currency because its the same always and else its unknown

In [172]:
data.drop(columns=['ship-country','currency'], inplace=True)

In [173]:
data.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'Amount', 'ship-city', 'ship-state',
       'ship-postal-code', 'promotion-ids', 'B2B', 'Unnamed: 22',
       'status_info', 'is_operational_loss'],
      dtype='object')

--------------------------Post Cleaning Analysis----------------------

Now, we will check for how many columns are required in the initial analysis.
['index', 'Order ID'(Ini), 'Date'(Ini), 'Status'(Pres), 'Fulfilment'(Ini)(Amazon or Merchant), 'Sales Channel (Ini)(Amazon or non-Amazon)',
       'ship-service-level(Ini)(Standard or Expedited)', 'Style(Ini)(Parent category of SKU for product idnetification)', 'SKU'(Ini)(sub-category of Style), 'Category (product identifier broad category, no descendant)', 'Size', 'ASIN'(Ini) (product catgeorisation similar to SKU with one exception),
       'Courier Status ([nan 'Shipped' 'Cancelled' 'Unshipped'])', ('Qty', 'currency', 'Amount')-> (Ini) {Econmomic Bundle}, 
       ('ship-city','ship-state', 'ship-postal-code', 'ship-country') -> (Ini) {Location Bundle}, 'promotion-ids (ini) (5787 unique values)','B2B(boolean)', 'fulfilled-by (Easy Ship or Nan)', 'Unnamed: 22' (False and NaNs)]

Now, we note down the negative metrics. Measures whose high frequency might indicate loss segment. 

fulfilled by is now dropped. now new column order and contents is 
Now, we will check for how many columns are required in the initial analysis.
['index', 'Order ID'(Ini), 'Date'(Ini), 'Status'(Pres), 'Fulfilment'(Ini)(Amazon or Merchant/Easy Ship), 'Sales Channel (Ini)(Amazon or non-Amazon)',
       'ship-service-level(Ini)(Standard or Expedited)', 'Style(Ini)(Parent category of SKU for product idnetification)', 'SKU'(Ini)(sub-category of Style), 'Category (product identifier broad category, no descendant)', 'Size', 'ASIN'(Ini) (product catgeorisation similar to SKU with one exception),
       'Courier Status ([nan 'Shipped' 'Cancelled' 'Unshipped'])', ('Qty', 'currency', 'Amount')-> (Ini) {Econmomic Bundle}, 
       ('ship-city','ship-state', 'ship-postal-code', 'ship-country') -> (Ini) {Location Bundle}, 'promotion-ids (ini) (5787 unique values)','B2B(boolean)', 'Unnamed: 22' (False and NaNs)]

In [174]:
print(data['is_operational_loss'].value_counts())
print(data['status_info'].value_counts())

is_operational_loss
0    108430
1     20539
Name: count, dtype: int64
status_info
shipped      78716
delivered    28769
cancelled    18424
returned      2109
unshipped      532
pending        413
lost             6
Name: count, dtype: int64


In [175]:
# ---------- 1. Define outcome: human cancellation ----------
data['status_info'] = data['status_info'].str.lower()

_tmp = data.copy()
_tmp['is_cancelled'] = _tmp['status_info'].isin(['cancelled', 'returned']).astype(int)


print("Status distribution:")
print(_tmp['status_info'].value_counts())
print("\nCancellation rate:", _tmp['is_cancelled'].mean())
print("-" * 60)


# ---------- 2. Helper function (local only) ----------
def cancellation_rate(df, col, min_count=50):
    out = (
        df.groupby(col)['is_cancelled']
          .agg(['mean', 'count'])
          .rename(columns={'mean': 'cancel_rate'})
          .reset_index()
          .sort_values('cancel_rate', ascending=False)
    )
    return out[out['count'] >= min_count]


# ---------- 3. Fulfilment & Sales Channel ----------
print("Fulfilment:")
display(cancellation_rate(_tmp, 'Fulfilment'))

print("Sales Channel:")
display(cancellation_rate(_tmp, 'Sales Channel '))


# ---------- 4. Fulfilment × Sales Channel ----------
_tmp['fulfilment_channel'] = (
    _tmp['Fulfilment'].astype(str) + ' | ' +
    _tmp['Sales Channel '].astype(str)
)

print("Fulfilment × Sales Channel:")
display(cancellation_rate(_tmp, 'fulfilment_channel', min_count=100))


# ---------- 5. Shipping & Quantity ----------
print("Ship service level:")
display(cancellation_rate(_tmp, 'ship-service-level'))

_tmp['qty_bucket'] = pd.cut(
    _tmp['Qty'],
    bins=[0, 1, 2, 5, _tmp['Qty'].max()],
    labels=['1', '2', '3–5', '6+'],
    include_lowest=True
)

print("Quantity bucket:")
display(cancellation_rate(_tmp, 'qty_bucket'))


# ---------- 6. Category, Size, State ----------
top_categories = _tmp['Category'].value_counts().head(8).index
print("Category (top 8):")
display(cancellation_rate(_tmp[_tmp['Category'].isin(top_categories)], 'Category'))

print("Size:")
display(cancellation_rate(_tmp, 'Size', min_count=100))

top_states = _tmp['ship-state'].value_counts().head(5).index
print("Ship state (top 5):")
display(cancellation_rate(_tmp[_tmp['ship-state'].isin(top_states)], 'ship-state'))


# ---------- 7. B2B (highly imbalanced, still explicit) ----------
print("B2B:")
display(
    _tmp.groupby('B2B')['is_cancelled']
        .agg(cancel_rate='mean', count='count')
        .reset_index()
)


# ---------- 8. Driver ranking ----------
drivers = []
for col in [
    'Fulfilment',
    'Sales Channel ',
    'fulfilment_channel',
    'ship-service-level',
    'qty_bucket',
    'B2B'
]:
    rates = _tmp.groupby(col)['is_cancelled'].mean()
    drivers.append({
        'feature': col,
        'delta_cancel_rate': rates.max() - rates.min()
    })

print("Driver ranking (by max–min cancellation rate):")
display(
    pd.DataFrame(drivers)
      .sort_values('delta_cancel_rate', ascending=False)
)


# ---------- 9. Cleanup ----------
del _tmp

Status distribution:
status_info
shipped      78716
delivered    28769
cancelled    18424
returned      2109
unshipped      532
pending        413
lost             6
Name: count, dtype: int64

Cancellation rate: 0.159208802115237
------------------------------------------------------------
Fulfilment:


,Fulfilment,cancel_rate,count
1,Merchant/Easy Ship,0.228378,39277
0,Amazon,0.128919,89692


Sales Channel:


,Sales Channel,cancel_rate,count
0,Amazon.in,0.159354,128845
1,Non-Amazon,0.008065,124


Fulfilment × Sales Channel:


...

,ship-state,cancel_rate,count
4,UTTAR PRADESH,0.173921,10637
3,TELANGANA,0.159488,11330
2,TAMIL NADU,0.153444,11483
1,MAHARASHTRA,0.146368,22259
0,KARNATAKA,0.141703,17325


B2B:


,B2B,cancel_rate,count
0,False,0.159643,128098
1,True,0.095293,871


Driver ranking (by max–min cancellation rate):


,feature,delta_cancel_rate
2,fulfilment_channel,0.220313
4,qty_bucket,0.162791
1,Sales Channel,0.151290
0,Fulfilment,0.099459
3,ship-service-level,0.093486
5,B2B,0.064351


## Interpretation: Why are orders getting cancelled or returned?

This analysis focuses on **customer-initiated order rejection**, combining **cancelled and returned** orders into a single outcome.  
All other outcomes are treated as the contrast group.

### Key observations

**1. Fulfilment method is a strong driver**  
Orders fulfilled by merchants show a substantially higher rejection rate (≈22.9%) compared to Amazon-fulfilled orders (≈12.9%).  
This indicates significantly higher customer drop-off in merchant fulfilment workflows, potentially driven by weaker delivery assurances, inventory mismatch, or lower trust.

---

**2. Sales channel shows the largest observed contrast (exploratory)**  
Orders placed via the Amazon channel exhibit a much higher rejection rate (≈15.9%) than Non-Amazon channels (≈0.8%).  
However, the Non-Amazon segment is very small (~124 orders), so this contrast is treated as **exploratory** and may reflect sampling or structural differences rather than a stable channel effect.

---

**3. Shipping speed is a critical differentiator**  
Standard shipping orders show a markedly higher rejection rate (≈22.4%) compared to expedited orders (≈13.0%).  

Notably, the expedited rejection rate remains nearly unchanged relative to cancellation-only analysis, while standard shipping exhibits a substantial increase.  
This suggests that **post-order rejection (including returns)** disproportionately affects standard shipments, whereas expedited shipments are primarily impacted only by pre-delivery cancellations.

---

**4. Order quantity is a dominant behavioral signal**  
Single-item orders have a high rejection rate (≈15.9%), while multi-item orders show substantially lower rates (≈10.6% for Qty = 2, with very limited volume beyond this).  
This pattern is consistent with lower customer commitment for single-item purchases.

---

**5. Order size exhibits a clear decreasing rejection trend**  
Smaller sizes (e.g., XS: ≈18.3%, S: ≈17.0%) are rejected more frequently than very large sizes (e.g., 6XL: ≈10.3%).  
The monotonic decline suggests that size correlates with customer certainty and fit confidence rather than category-specific effects.

---

**6. B2B orders are more stable**  
B2B orders show a lower rejection rate (≈9.5%) compared to non-B2B orders (≈16.0%).  
While B2B volume is smaller, the direction is consistent with more deliberate and planned purchasing behavior.

---

## Strategy & Next Steps

Strategies are proposed only where there is a clear, measured association between an operational lever and customer rejection behavior.

### Intervention-ready strategies

**1. Reduce rejection in merchant-fulfilled orders**
- Tighten delivery promise communication for merchant fulfilment
- Delay payment capture until shipment confirmation
- Actively monitor merchant-specific rejection rates

**Metric:** Rejection rate (cancelled + returned) for merchant fulfilment

---

**2. Address elevated rejection in standard shipping**
- Incentivize expedited shipping for high-risk segments (e.g. single-item orders)
- Surface delivery timelines more prominently during checkout
- Consider auto-upgrades for high-value or high-risk orders

**Metric:** Rejection rate by shipping speed

---

**3. Target high-risk single-item orders**
- Introduce bundle or add-on incentives
- Apply stronger confirmation nudges before final order placement

**Metric:** Rejection rate for Qty = 1

---

### Investigation-required strategies

**4. Validate sales channel effects**  
The extreme contrast between Amazon and Non-Amazon channels requires further validation before action.

Next steps:
- Profile Non-Amazon orders by fulfilment, quantity, and B2B status
- Recompute rejection rates within matched segments
- Reassess once Non-Amazon volume increases

**Metric:** Stratified rejection rate by sales channel

---

### Defensive baseline

**5. Use B2B orders as a control segment**  
Given their lower and more stable rejection rates, B2B orders should be excluded from mitigation experiments and used as a behavioral baseline.

---

## Summary

Customer rejection (cancelled + returned) is primarily driven by fulfilment method, shipping speed, and order commitment signals such as quantity and size.  
Standard shipping shows substantial additional rejection beyond pre-delivery cancellations, while expedited shipping remains stable.  
Although sales channel exhibits the largest observed contrast, its interpretation is constrained by very low support in the Non-Amazon segment.

Future work should focus on targeted interventions for high-confidence drivers while validating high-contrast but low-support findings.


since cancelled + returned covers most of the true cases for is_operational loss. loss segment analysis should look identical to this.

In [176]:
data['Courier Status'].fillna('Operation Finished', inplace=True)
data['Courier Status'].unique()

array(['Operation Finished', 'shipped', 'cancelled', 'unshipped'],
      dtype=object)

In [177]:
import automated_pipeline as ap
ap.terminology_distinction_check(data, 'Fulfilment', 'Amazon', 'Merchant/Easy Ship', 'Courier Status')

(('Amazon', {'cancelled'}), ('Merchant/Easy Ship', {'Operation Finished'}))

Downstream is merely for practice and not to aid analysis

In [178]:
hhhh

NameError: name 'hhhh' is not defined

Now, we check for unnamed: 22 column and what that means in terms of the other columns.

In [ ]:
# Binary target: 1 if Unnamed: 22 is present, 0 if NaN
y = data["Unnamed: 22"].notna().astype(int)

y.value_counts()
# Full single-cell pipeline: test whether Unnamed: 22 is predictable from all other columns

In [ ]:
suspect_cols = []

for c in data.columns:
    if c == "Unnamed: 22":
        continue
    p1 = data.loc[data["Unnamed: 22"].notna(), c].isna().mean()
    p0 = data.loc[data["Unnamed: 22"].isna(), c].isna().mean()
    if abs(p1 - p0) > 0.01:   # tunable threshold
        suspect_cols.append(c)

suspect_cols

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

# ----------------------------
# Target
# ----------------------------
y = data["Unnamed: 22"].notna().astype(int)

# ----------------------------
# Features
# ----------------------------
X = data.drop(columns=["Unnamed: 22", "Order ID", "Date"])

cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(exclude="object").columns.tolist()

X[cat_cols] = X[cat_cols].astype(str)

# ----------------------------
# Train / test split
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ----------------------------
# Preprocessing (KEEP SPARSE)
# ----------------------------
preprocess = ColumnTransformer(
    transformers=[
        (
            "cat",
            Pipeline([
                ("imputer", SimpleImputer(strategy="constant", fill_value="__NaN__")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))  # sparse=True default
            ]),
            cat_cols
        ),
        (
            "num",
            Pipeline([
                ("imputer", SimpleImputer(strategy="constant", fill_value=0))
            ]),
            num_cols
        )
    ]
)

# ----------------------------
# Model (SPARSE-SAFE)
# ----------------------------
model = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(
        solver="saga",
        max_iter=2000,
        n_jobs=-1
    ))
])

# ----------------------------
# Fit + evaluate
# ----------------------------
model.fit(X_train, y_train)

print("Base rate:", y.mean())
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy :", model.score(X_test, y_test))